In [2]:
import pandas as pd

stock = pd.read_csv('../../data/explore/stock_url_list.csv')
stock.head()

,Stock,ราคาล่าสุด(Home),Sector,กรรมการบริษัท,งบการเงิน,บริษัท/หลักทรัพย์,ผู้ถือหุ้นรายใหญ่,ราคาย้อนหลัง
0,CHOTI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...
1,EE,/C13_FastQuote_Main.jsp?txtSymbol=EE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...
2,GFPT,/C13_FastQuote_Main.jsp?txtSymbol=GFPT,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...
3,LEE,/C13_FastQuote_Main.jsp?txtSymbol=LEE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...
4,STA,/C13_FastQuote_Main.jsp?txtSymbol=STA,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...


# งบดุล

$$\text{Asset} = \text{Equity} + \text{Debt}$$


ข้อมูลที่ต้องการได้แก่ (กรณีดึงจาก settrade.com)

### งบดุล
- หนี้สินรวม (2017,2016,...) - Home
- เจ้าหนี้การค้าและเจ้าหนี้อื่น (2017,2016,...) - งบเต็ม
- ส่วนของผู้ถือหุ้น (2017,2016,...) - Home
- หนี่สินที่มีภาระดอกเบี้ย (2017,2016,...) - งบเต็ม ซึ่งประกอบไปด้วย
    - เงินเบิกเกินบัญชีและเงินกู้ยืมระยะสั้นจากสถาบันการเงิน(งบเต็ม)
    - ส่วนของหนี้สินระยะยาวที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    - หนี้สินระยะยาว-สุทธิจากส่วนที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    
### ตรวจสอบส่วนของผู้ถือหุ้นว่ามาจากกำไรหรือระดมทุนเพิ่ม
- มูลค่าหุ้นที่เรียกชำระแล้ว (2017,2016,2015,...) - Home

ข้อมูลจากตาราง stock ทำให้เรารู้ url ของข้อมูลที่เราจะไปดึงมาแล้ว ดังนั้นทำฟังก์ชั่นสำหรับดึงข้อมูลงบการเงิน จะได้

In [315]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [351]:
def is_float(input):
    try:
        num = float(input)
    except ValueError:
        return False
    return True

In [414]:
def getTableHeadData(head):
    data_head = head.find_all('strong')
    date_html = data_head[1:len(data_head)]
    date_text = [ list(html.stripped_strings) for html in date_html ]
    dates = [ year_date[1] for year_date in date_text if len(year_date) > 0 ]
    return [data_head[0].text,dates]

In [456]:
def getTableSecondHeadData(head):
    data_head = head.find_all('strong')
    date_html = data_head[1:len(data_head)]
    date_text = [ list(html.stripped_strings) for html in date_html ]
    dates = [ year_date[0] for year_date in date_text if len(year_date) > 0 ]
    return [data_head[0].text,dates]

In [457]:
def getTableRowData(row):
    data_row = row.find_all('td')
    data_list = [ list(data.stripped_strings) for data in data_row if len(data) > 0 ]
    data_text = [ data[0].replace(',','') for data in data_list if len(data) > 0 ]
    datas = [ float(data) for data in data_text if is_float(data) ]
    return [data_text[0],datas]

In [458]:
def getTableRowText(row):
    data_row = row.find_all('td')
    data_list = [ list(data.stripped_strings) for data in data_row if len(data) > 0 ]
    data_text = [ data[0] for data in data_list if len(data) > 0 ]
    return [data_text[0],data_text[1:len(data_text)]]

In [459]:
def getStockDetail(url,stock):
    page = requests.get(url) # ดูดหน้าเว็บมา
    
    if page.status_code == 200 :
        # All HTML of Page
        soup = BeautifulSoup(page.content, "html5lib")
        # ซอยข้อมูลทั้ง page เป็นทีละส่วนๆ แล้วลงลึกไปเอาข้อมูลทีละจุดๆ
        html = list(soup.children)[1]
        head = list(html.children)[0]
        body = list(html.children)[2]
        
        # ***
        table_info = list(body.find(class_='table-info'))
        table_header1 = table_info[3]
        table_body1 = table_info[5]
        table_body_row1 = table_body1.find_all('tr')
        table_header2 = table_info[6]
        table_body2 = table_info[8]
        table_body_row2 = table_body2.find_all('tr')

        # -------------- Top Table ----------------
        # Date : งวดงบการเงิน ณ วันที่
        dates = getTableHeadData(table_header1)
        print(dates[0],dates[1])
        
        # Asset : สินทรัพย์รวม
        asset = getTableRowData(table_body_row1[1])
        print(asset[0],asset[1])
        
        # Debt : หนี้สินรวม
        debt = getTableRowData(table_body_row1[2])
        print(debt[0],debt[1])
        
        # Equity : ส่วนของผู้ถือหุ้น
        equity = getTableRowData(table_body_row1[3])
        print(equity[0],equity[1])
        
        # Paid-up Capital : มูลค่าหุ้นที่เรียกชำระแล้ว
        paid_up = getTableRowData(table_body_row1[4])
        print(paid_up[0],paid_up[1])
        
        # Revenue : รายได้รวม
        revenue = getTableRowData(table_body_row1[5])
        print(revenue[0],revenue[1])
        
        # Net Profit : กำไรสุทธิ
        net_profit = getTableRowData(table_body_row1[6])
        print(net_profit[0],net_profit[1])
        
        # EPS (Baht) : กำไรต่อหุ้น (บาท)
        eps = getTableRowData(table_body_row1[7])
        print(eps[0],eps[1])
        
        # ROA(%)
        roa = getTableRowData(table_body_row1[9])
        print(roa[0],roa[1])
        
        # ROE(%)
        roe = getTableRowData(table_body_row1[10])
        print(roe[0],roe[1])
        
        # Net Profit Margin(%) : อัตรากำไรสุทธิ(%)
        net_profit_margin = getTableRowData(table_body_row1[11])
        print(net_profit_margin[0],net_profit_margin[1])

        # -------------- Bottom Table ----------------
        # Date : ค่าสถิติสำคัญ ณ วันที่
        dates2 = getTableSecondHeadData(table_header2)
        print(dates2[0],dates2[1])
        
        # Last Price(Baht) : ราคาล่าสุด(บาท)
        last_price = getTableRowData(table_body_row2[0])
        print(last_price[0],last_price[1])
        
        # Market Cap. : มูลค่าหลักทรัพย์ตามราคาตลาด
        market_cap = getTableRowData(table_body_row2[1])
        print(market_cap[0],market_cap[1])
        
        # F/S Period (As of date) : วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ
        f_s_period = getTableRowText(table_body_row2[2])
        print(f_s_period[0],f_s_period[1])
        
        # P/E (เท่า)
        p_e = getTableRowData(table_body_row2[3])
        print(p_e[0],p_e[1])
        
        # P/BV (เท่า)
        pbv = getTableRowData(table_body_row2[4])
        print(pbv[0],pbv[1])
        
        # Book Value per share (Baht) : มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท)
        book_value_per_share = getTableRowData(table_body_row2[5])
        print(book_value_per_share[0],book_value_per_share[1])
        
        # Dvd. Yield(%) : อัตราส่วนเงินปันผลตอบแทน(%)
        dvd_yield = getTableRowData(table_body_row2[6])
        print(dvd_yield[0],dvd_yield[1])
        
        
#         previous_price_url = nav_menu[3].find('a').get('href')
#         company_stock_name = nav_menu[5].find('a').text #บริษัท/หลักทรัพย์
#         company_stock_url = nav_menu[5].find('a').get('href') 
#         board_of_director_name = nav_menu[7].find('a').text #กรรมการบริษัท
#         board_of_director_url = nav_menu[7].find('a').get('href') 
#         major_share_holder_name = nav_menu[9].find('a').text #ผู้ถือหุ้นรายใหญ่
#         major_share_holder_url = nav_menu[9].find('a').get('href')  
#         financial_name = nav_menu[11].find('a').text #งบการเงิน
#         financial_url = nav_menu[11].find('a').get('href')

#         # Convert to DataFrame         
#         stock_df = pd.DataFrame({previous_price_name: [previous_price_url], 
#                                  company_stock_name: [company_stock_url],
#                                  board_of_director_name: [board_of_director_url],
#                                  major_share_holder_name: [major_share_holder_url],
#                                  financial_name: [financial_url]
#                                 })
#         stock_df['Stock'] = stock
#         return equitys
    else:
        print("Can't get content from this URL!!!")

In [460]:
prefix_url = 'https://www.settrade.com'
suffix_url = '/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageId=13&selectPage=6'
settrade_url = prefix_url + suffix_url

In [461]:
xx = getStockDetail(settrade_url,'LEE')
xx

งวดงบการเงิน ณ วันที่ ['31/12/2557', '31/12/2558', '31/12/2559', '31/12/2560']
สินทรัพย์รวม [2887.18, 2827.41, 2995.53, 2989.21]
หนี้สินรวม [337.54, 288.57, 291.68, 322.47]
ส่วนของผู้ถือหุ้น [2525.15, 2513.0, 2676.55, 2638.26]
มูลค่าหุ้นที่เรียกชำระแล้ว [922.18, 922.18, 922.18, 922.18]
รายได้รวม [3834.74, 3505.79, 3478.95, 3368.1]
กำไรสุทธิ [177.44, 145.56, 248.14, 191.14]
กำไรต่อหุ้น (บาท) [0.19, 0.16, 0.27, 0.21]
ROA(%) [6.97, 5.18, 10.21, 7.57]
ROE(%) [6.99, 5.78, 9.56, 7.19]
อัตรากำไรสุทธิ(%) [4.63, 4.15, 7.13, 5.67]
ค่าสถิติสำคัญ ณ วันที่ ['30/12/2557', '30/12/2558', '30/12/2559', '29/12/2560', '26/04/2561']
ราคาล่าสุด(บาท) [2.88, 2.62, 3.1, 3.08, 2.9]
มูลค่าหลักทรัพย์ตามราคาตลาด [2655.88, 2416.11, 2858.76, 2840.32, 2674.32]
วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ ['30/09/2557', '30/09/2558', '30/09/2559', '30/09/2560', '31/12/2560']
P/E (เท่า) [14.62, 16.68, 11.82, 13.67, 13.99]
P/BV (เท่า) [1.08, 0.98, 1.1, 1.09, 1.01]
มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท) [2.68, 2.66, 2.82, 2.82, 2.86]

In [175]:
len(xx[0])

10